## 配置环境

运行下列命令配置一个名称为 `cobra` 的 Conda 环境，并在其中安装需要运行的软件或模块。
`cometspy` 就是 COMETS 的 Python 版本。


> **关于 COMETS**
>
> COMETS - Computation Of Microbial Ecosystems in Time and Space.
>
> COMETS is a software platform for performing computer simulations of spatially structured microbial communities. It is based on stoichiometric modeling of the genome-scale metabolic network of individual microbial species using dynamic flux balance analysis, and on a discrete approximation of diffusion. For more information, see Harcombe et al., Metabolic Resource Allocation in Individual Microbes Determines Ecosystem Interactions and Spatial Dynamics, Cell Reports, 2014.
> 
> COMETS is built and maintained by the Daniel Segre Lab at Boston University.

```bash
conda create -n cobra
conda activate cobra
conda install matplotlib
pip install cometspy
```

在运行 Jupyter Notebook 时，使用该 Conda 环境作为解释器。

Prerequisites for COMETS installation are to have Java 64-bit and Gurobi installed.

* Java can be downloaded and installed from: https://www.oracle.com/technetwork/java/javase/downloads/index.html or https://java.com/en/download/manual.jsp. The minimum version of   required 64-bit Java is 1.8.
* Gurobi can be downloaded and installed from: http://www.gurobi.com/downloads/gurobi-optimizer-eula/. The installation of Gurobi requires obtaining a license. Free academic license can be obtained from:  https://www.gurobi.com/downloads/end-user-license-agreement-academic/.



## 导入模块


In [1]:
import cometspy as c
import cobra
from os.path import join
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# 加载模型
model_dir = "model"
eco_model = cobra.io.load_json_model(join(model_dir, "iML1515.json"))
ppu_model = cobra.io.load_json_model(join(model_dir, "iJN1463.json"))

#去除交换反应边界
for i in eco_model.reactions:
    if 'EX_' in i.id:
        i.lower_bound=-1000.0

for i in ppu_model.reactions:
    if 'EX_' in i.id:
        i.lower_bound=-1000.0

eco=c.model(eco_model)
ppu=c.model(ppu_model)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-02


/opt/homebrew/anaconda3/envs/cobra/lib/python3.12/site-packages/cometspy/model.py:623: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.smat = pd.concat([self.smat, cdf])
/opt/homebrew/anaconda3/envs/cobra/lib/python3.12/site-packages/cometspy/model.py:636: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
/opt/homebrew/anaconda3/envs/cobra/lib/python3.12/site-packages/cometspy/model.py:623: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. T

In [3]:
# 打开交换反应，以便外细胞代谢物浓度和米氏动力学控制吸收界限
eco.open_exchanges()
ppu.open_exchanges()

eco.obj_style="MAX_OBJECTIVE_MIN_TOTAL" # 默认的FBA选项是“最大化目标通量”
ppu.obj_style="MAX_OBJECTIVE_MIN_TOTAL"

In [4]:
#eco
eco.change_bounds('EX_ca2_e',-1000,1000) 
eco.change_bounds('EX_cl_e',-1000,1000)
eco.change_bounds('EX_h_e',-1000,1000) 
eco.change_bounds('EX_co2_e',-1000,1000) 
eco.change_bounds('EX_h2o_e',-1000,1000)
eco.change_bounds('EX_k_e',-1000,1000) 
eco.change_bounds('EX_mg2_e',-1000,1000) 
eco.change_bounds('EX_na1_e',-1000,1000) 
eco.change_bounds('EX_o2_e',-1000,1000) 
eco.change_bounds('EX_so4_e',-1000,1000) 
eco.change_bounds('EX_nh4_e',-1000,1000) 
eco.change_bounds('EX_pi_e',-1000,1000) 
eco.change_bounds('EX_cobalt2_e',-1000,1000) 
eco.change_bounds('EX_malt_e',-10,1000) 

In [5]:
#ppu
ppu.change_bounds('EX_ca2_e',-1000,1000) 
ppu.change_bounds('EX_cl_e',-1000,1000)
ppu.change_bounds('EX_h_e',-1000,1000) 
ppu.change_bounds('EX_co2_e',-1000,1000) 
ppu.change_bounds('EX_h2o_e',-1000,1000)
ppu.change_bounds('EX_k_e',-1000,1000) 
ppu.change_bounds('EX_mg2_e',-1000,1000) 
ppu.change_bounds('EX_na1_e',-1000,1000) 
ppu.change_bounds('EX_o2_e',-1000,1000) 
ppu.change_bounds('EX_so4_e',-1000,1000) 
ppu.change_bounds('EX_nh4_e',-1000,1000) 
ppu.change_bounds('EX_pi_e',-1000,1000) 
ppu.change_bounds('EX_cobalt2_e',-1000,1000) 
ppu.change_bounds('EX_malt_e',-10,1000) 

reaction couldnt be found


In [6]:
# 1-1混菌培养
eco.initial_pop=[0,0,3.9e-7]
ppu.initial_pop=[0,0,3.9e-7]

test_tube_malt = c.layout()
# 向试管布局中添加模型
test_tube_malt.add_model(eco)
test_tube_malt.add_model(ppu)

building empty layout model
models will need to be added with layout.add_model()


/opt/homebrew/anaconda3/envs/cobra/lib/python3.12/site-packages/cometspy/layout.py:1361: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.media = pd.concat([self.media, new_row],


In [7]:
# 麦芽糖底物设置
test_tube_malt.set_specific_metabolite('ca2_c', 1000) 
test_tube_malt.set_specific_metabolite('cl_c', 1000) 
test_tube_malt.set_specific_metabolite('h_c', 1000) 
test_tube_malt.set_specific_metabolite('co2_c', 1000) 
test_tube_malt.set_specific_metabolite('h2o_c', 1000) 
test_tube_malt.set_specific_metabolite('k_c', 1000) 
test_tube_malt.set_specific_metabolite('mg2_c', 1000) 
test_tube_malt.set_specific_metabolite('na1_c', 1000) 
test_tube_malt.set_specific_metabolite('o2_c', 1000) 
test_tube_malt.set_specific_metabolite('so4_c', 1000)
test_tube_malt.set_specific_metabolite('nh4_c', 1000) 
test_tube_malt.set_specific_metabolite('pi_c', 1000) 
test_tube_malt.set_specific_metabolite('cobalt2_c', 1000) 
test_tube_malt.set_specific_metabolite('malt_c', 5.00) 

In [8]:
import os
os.environ['GUROBI_HOME'] = '/Library/gurobi1000/macos_universal2/'
#os.environ['COMETS_HOME'] = '/Users/gaoch/GitHub/comets/comets_simplified'
os.environ['COMETS_HOME'] = '/Users/gaoch/Downloads/comets_2.10.5_linux'


In [9]:
# 创建并设置COMETS参数
comp_params = c.params()
comp_params.set_param('defaultVmax', 10)  # 默认最大反应速率
comp_params.set_param('defaultKm', 0.01)  # 默认米氏常数
comp_params.set_param('exchangestyle', 'Monod Style')  # 交换风格为Monod模型
comp_params.set_param('defaultHill', 1)  # 默认希尔系数

# 设置日志记录参数
comp_params.set_param('writeMediaLog', True)  # 是否写入培养基日志
comp_params.set_param('writeFluxLog', True)  # 是否写入通量日志
comp_params.set_param('MediaLogRate', 1)  # 培养基日志记录频率
comp_params.set_param('FluxLogRate', 1)  # 通量日志记录频率
comp_params.set_param('writeTotalBiomassLog', True)  # 是否写入总生物量日志
comp_params.set_param('writeBiomassLog', True)  # 是否写入生物量日志
comp_params.set_param('totalBiomassLogRate', 1)  # 总生物量日志记录频率
comp_params.set_param('BiomassLogRate', 1)  # 生物量日志记录频率
comp_params.set_param("writeSpecificMediaLog", True)  # 是否写入特定培养基日志
comp_params.set_param("specificMediaLogRate", 1)  # 特定培养基日志记录频率
comp_params.set_param("specificMedia", "cpd00027_e0")  # 特定培养基的化合物ID
# 运行模拟
#sim = c.comets(test_tube_malt, comp_params)
#sim.run()
#one_cell=3.9e-13 # gram dry weight of a cell

comp_assay=c.comets(test_tube_malt, comp_params)
comp_assay.run()




Running COMETS simulation ...
Error: COMETS simulation did not complete

     examine comets.run_output for the full java trace

     if we detect a common reason, it will be stated in the RuntimeError at the bottom


RuntimeError: COMETS simulation did not complete:
 undetected reason. examine comets.run_output for JAVA trace

In [10]:
print(comp_assay.run_output)

-script
running script file: /Users/gaoch/GitHub/merge_model/.current_script_0x17a28c410
Current Java version: 1.8.0_401
Parameters file .current_global_0x17a28c410 loaded.
Parameters file .current_package_0x17a28c410 loaded.
Loading layout file '.current_layout_0x17a28c410'...
null/COMETS_manifest.txt (No such file or directory)
Unable to initialize manifest file. 
Continuing without writing manifest file.
Found 2 model files!
Loading './iML1515.cmd' ...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-02
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-02
Done!
 Testing default parameters...
Done!
Optimizer status code = 5 (looks ok!)
objective solution = 74.895719394727
Loading './iJN1463.cmd' ...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-02
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-02
Done!
 Testing default paramet

In [12]:
# 获取并分析结果
biomass_data = comp_assay.get_biomass_data()
flux_data = comp_assay.get_flux_data()

AttributeError: 'comets' object has no attribute 'get_biomass_data'